In [4]:
import os
import pymysql
import mysql.connector
from tabulate import tabulate
import getpass
import re

host=os.getenv("MYSQL_HOST")
port=os.getenv("MYSQL_PORT")
user=os.getenv("MYSQL_USER")
password=os.getenv("MYSQL_PASSWORD")
database=os.getenv("MYSQL_DATABASE")

conn=pymysql.connect(
    host=host,
    port=int(3306),
    user="root",
    password="root",
    db="project",
    charset="utf8mb4")
cur=conn.cursor()

q='use project'
cur.execute(q)

def oper():
    n=int(input('''
                    Please Enter the option 
                           1.Sell
                           2.Buy
                           3.Update
                           4.Delete
                           5.View your properties
                           6.Exit
                       '''))
    if n==1:
        sell()
    elif n==2:
        buy()
    elif n==3:
        update()
    elif n==4:
        delete()
    elif n==5:
        view()
    elif n==6:
        print("Thank you for using PropertyGuru")
    else:
        print("Enter valid option")
    






def view():
    email=input("Enter your mail ID")
    passw=getpass.getpass(prompt="Enter the password: ")
    sql="select plot_id,property_name,property_type,property_place,property_price,contact_num from plots where p_email= %s and pass= %s"
    data=(email,passw)
    cur=conn.cursor()
    cur.execute(sql,data)
    field3=cur.fetchall()
    if field3!=():
        print(tabulate(field3,headers=['Plot_ID','Property_Name','Property_Type','Property_place','Property Price','Contact_Number']))
        oper()
    else:
        print("No Property belongs to the entered email or password,Please check once again")
        oper()






def update():
    email_id=input("Enter the email id to update your record:  ")
    passw=getpass.getpass(prompt="Enter the password: ")
    p_id=input("Enter the Plot Id you like to update: ")
    sql="select p_email,pass,plot_id from plots where p_email= %s and pass= %s and plot_id=%s"
    data=(email_id,passw,p_id)
    cur=conn.cursor()
    cur.execute(sql,data)
    field1=cur.fetchone()
    if field1 is not None:
        if email_id==str(field1[0]):
            
            sql='update plots set PROPERTY_NAME = %s,PROPERTY_TYPE = %s,PROPERTY_PLACE=%s,Property_Price=%s,CONTACT_NUM=%s where p_email=%s and plot_id=%s'
            name=input("Enter the property name: ")
            typ=input("Enter the property type: ")
            place=input("Enter the place: ")
            p_price=int(input("Enter the price: "))
            cont=int(input("Enter the contact number: "))
            data=(name,typ,place,p_price,cont,email_id,p_id)
            cur.execute(sql,data)
            conn.commit()
            print("Update done")
            oper()
    else:
        print("Invalid email_id or password or plot id")
        print("------------------------------------")
        update()

def delete():
    email_id=input("Enter the Email id to delete your record: ")
    passw=getpass.getpass(prompt="Enter the password: ")
    plot_id=input("Enter the Plot Id: ")
    sql="select p_email,pass,plot_id from plots where p_email= %s and pass= %s and plot_id=%s"
    data=(email_id,passw,plot_id)
    cur=conn.cursor()
    cur.execute(sql,data)
    field1=cur.fetchone()
    if field1 is not None:
        if email_id==str(field1[0]):
            sql="delete from plots where p_email= %s and pass=%s and plot_id=%s"
            data=(email_id,passw,plot_id)
            cur=conn.cursor()
            cur.execute(sql,data)
            conn.commit()
            print("Deleting done")
            oper()
    else:
        print("Invalid email_id or password or Plot ID")
        print("------------------------------------")
        delete()
        
def sell():
    sql='INSERT INTO plots(PROPERTY_NAME,PROPERTY_TYPE,p_email,pass,PROPERTY_PLACE,Property_Price,CONTACT_NUM) VALUES(%s,%s,%s,%s,%s,%s,%s)'
    name=input("Enter the property name: ")
    typ=input("Enter the property type: ")
    p_email=input("Enter the Email id: ")
    passw=getpass.getpass(prompt="Enter the password: ")
    place=input("Enter the place: ")
    p_price=int(input("Enter the price: "))
    cont=int(input("Enter the contact number: "))
    data=(name,typ,p_email,passw,place,p_price,cont)
    cur=conn.cursor()
    cur.execute(sql,data)
    conn.commit()
    print("Done Uploading")
    print("------------------------------------")
    oper()

    
def buy():
    p=input("Enter the place you are searching for: ")
    sql='select plot_id,property_name,property_type,property_place,property_price,contact_num from plots where property_place= %s'
    data=p
    cur=conn.cursor()
    cur.execute(sql,data)
    field2=cur.fetchall()
    if field2!=():
        print(tabulate(field2,headers=['Plot_ID','Property_Name','Property_Type','Property_place','Property Price','Contact_Number']))
        oper()
    else:
        print("Sorry,No properties in your preferred location as of now.\n Sorry for inconvenience")
        print("------------------------------------")
        oper()

def email_val():
    global email
    email=input("Enter your email id : ")
    if(re.fullmatch(r'[A-Za-z0-9._+-]+@[A-Za-z0-9,-]+\.[A-Z|a-z]+',email)):
        return True
        
def password_validation(passw):
    if re.match(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*[@$!%*?&])(?=.*\d)[a-zA-Z\d@$!%*?&]{8,}$",passw):
        return True
def num_val():
    global num
    num=input("Enter your contact number : ")
    if(re.fullmatch('[0-9]+',num) and len(num)==10):
        return True
    
    
def signup():
    xy=email_val()
    if xy==True:
        global passw
        passw=getpass.getpass(prompt='''
        Password Instruction:
        MUST contain at least 8 characters (8+ recommended)
        MUST contain at least one uppercase letter
        MUST contain at least one lowercase letter
        MUST contain at least one number
        MUST contain at least one special character (I'#$%8 0*+,-/:<=>?@N^_ (I)~)
        Enter the password''')
        sql="select email_id,pass from PropertyGurus where email_id= %s"
        data=(email)
        cur=conn.cursor()
        cur.execute(sql,data)
        field2=cur.fetchone()
        if field2 is not None:
            print("Email_Id Already Exist,Please try again")
            print("------------------------------------")
            welcome()
        else:
            x=password_validation(passw)
            if x==True:
                if num_val()==True:
                    sql='INSERT INTO PROPERTYGURUS(FULL_NAME,EMAIL_ID,PASS,CON_NUM,PLACE) VALUES(%s,%s,%s,%s,%s)'
                    fname=input("enter your full name : ")
                    place=input("enter your place name : ")
                    data=(fname,email,passw,num,place)
                    cur.execute(sql,data)
                    conn.commit()
                    oper()
                else:     
                    print("Please enter Valid contact number")
                    signup()    
            else:
                print("Password is too weak,try once again")
                print("------------------------------------")
                signup()
    else:
        print("Enter Proper Email_id")
        signup()
        
        
def login():
    email_id=input("Enter emailid:  ")
    passw=getpass.getpass(prompt="Enter the password: ")
    sql="select email_id,pass from PropertyGurus where email_id= %s and pass= %s"
    data=(email_id,passw)
    cur=conn.cursor()
    cur.execute(sql,data)
    field1=cur.fetchone()
    if field1 is not None:
        if email_id==str(field1[0]):
            print("Logged in ")
            n=int(input('''
            Please Enter the option 
                           1.Sell
                           2.Buy
                           3.Update
                           4.Delete
                           5.View your properties
                       '''))
            if n==1:
                sell()
            elif n==2:
                buy()
            elif n==3:
                update()
            elif n==4:
                delete()
            elif n==5:
                view()
            else:
                print("Enter valid option")
        
    else:
        print("Invalid Password or Emailid")
        print("------------------------------------")
        login()
       
    

def welcome():
    print("Welcome to PropertyGuru")
    print("      -Your Property,Our Priority")
    print("1.Signup")
    print("2.Login")
    n=int(input("Enter your Choice"))
    if n==1:
        signup()
    elif n==2:
        login()
    else:
        print("Enter Valid Number")
        print("------------------------------------")
        welcome()
        
welcome()



Welcome to PropertyGuru
      -Your Property,Our Priority
1.Signup
2.Login
Enter your Choice1
Enter your email id : gvr
Enter Proper Email_id
Enter your email id : erc
Enter Proper Email_id
Enter your email id : er
Enter Proper Email_id
Enter your email id : pm@gmail.com

        Password Instruction:
        MUST contain at least 8 characters (8+ recommended)
        MUST contain at least one uppercase letter
        MUST contain at least one lowercase letter
        MUST contain at least one number
        MUST contain at least one special character (I'#$%8 0*+,-/:<=>?@N^_ (I)~)
        Enter the password········
Email_Id Already Exist,Please try again
------------------------------------
Welcome to PropertyGuru
      -Your Property,Our Priority
1.Signup
2.Login
Enter your Choice1
Enter your email id : hari@gmail.com

        Password Instruction:
        MUST contain at least 8 characters (8+ recommended)
        MUST contain at least one uppercase letter
        MUST contain at lea

KeyboardInterrupt: Interrupted by user